# Chapter 15: Kubernetes Networking

Having mastered Kubernetes core resources in Chapter 14, we now examine the networking fabric that binds these components together. Networking in Kubernetes differs fundamentally from traditional Docker networking or virtual machine networking. The platform imposes specific requirements—namely that every Pod receives a unique IP address across the cluster and that all Pods can communicate without Network Address Translation (NAT)—which necessitate specialized Container Network Interface (CNI) implementations.

This chapter explores the Kubernetes networking model from Pod-level communication through cluster-wide service discovery, ingress routing, and security policies. Understanding these mechanisms is essential for debugging connectivity issues, designing secure microservices architectures, and optimizing application performance in distributed environments.

## 15.1 Pod Networking Model

The Kubernetes networking model establishes fundamental principles that distinguish it from other container orchestration systems. These requirements ensure transparent, flat networking across the cluster.

### Core Networking Requirements

Kubernetes mandates three essential networking behaviors:

1. **All containers (Pods) can communicate with all other containers without NAT**
   - Every Pod has a unique IP routable within the cluster
   - Source IP seen by the destination matches the sender's IP
   - No port mapping or host networking required for basic connectivity

2. **All nodes can communicate with all containers (and vice versa) without NAT**
   - Kubelet can reach Pods directly for health checks and exec operations
   - Pods can reach node IPs for host-based services

3. **The IP that a container sees itself as is the same IP that others see it as**
   - No internal/external IP dichotomy within the cluster
   - Self-identification matches external identification

### Network Namespace Implementation

Each Pod operates within its own network namespace on the host node, connected to the cluster network through a virtual ethernet (veth) pair:

**Architecture Flow:**
1. **Pod Namespace**: Contains the container's eth0 interface
2. **Veth Pair**: Virtual cable connecting Pod namespace to host namespace
3. **CNI Bridge/Interface**: Connects to the node's physical network or overlay
4. **Routing/Fabric**: Routes packets between nodes via underlay or overlay networks

**IP Address Allocation:**
- The CNI plugin assigns IP addresses to Pods from a per-node CIDR range
- The node CIDRs aggregate into the cluster CIDR (e.g., 10.244.0.0/16)
- IPAM (IP Address Management) ensures uniqueness across the cluster

### Cluster DNS

Every Pod receives DNS configuration enabling service discovery:

- **DNS Server**: CoreDNS (or kube-dns in older clusters) runs as a cluster add-on
- **Search Domains**: Configured with default search paths:
  - `default.svc.cluster.local`
  - `svc.cluster.local`
  - `cluster.local`
- **NDOTS**: Default ndots:5 means names with fewer than 5 dots get search domain suffixes appended

**Verification:**
```bash
# Inside a Pod
cat /etc/resolv.conf
# Output:
# nameserver 10.96.0.10
# search default.svc.cluster.local svc.cluster.local cluster.local
# options ndots:5
```

## 15.2 Service DNS

While Services provide stable IP addresses (ClusterIPs), DNS provides stable names that abstract away IP changes during Pod restarts or scaling events.

### CoreDNS Architecture

CoreDNS is the default cluster DNS server, replacing the legacy kube-dns implementation. It runs as a Deployment (typically 2+ replicas for high availability) and Service within the `kube-system` namespace.

**DNS Record Types:**

**A/AAAA Records (Normal Services):**
```
<service-name>.<namespace>.svc.<cluster-domain>
```
Example: `mysql.production.svc.cluster.local` resolves to the Service ClusterIP (e.g., 10.96.123.45)

**SRV Records (Named Ports):**
```
_<port-name>._<protocol>.<service-name>.<namespace>.svc.<cluster-domain>
```
Example: `_http._tcp.web.production.svc.cluster.local`

**PTR Records (Reverse DNS):**
Map IPs back to service names for logging and authentication purposes.

**Headless Services:**
For Headless Services (clusterIP: None), DNS returns A records for each Pod IP directly rather than the Service IP:
```
pod-ip-address.<service-name>.<namespace>.svc.<cluster-domain>
```
Example: `10-244-1-5.db-headless.production.svc.cluster.local`

### DNS Resolution Patterns

**Same Namespace Resolution:**
```bash
# From a Pod in the 'production' namespace
curl http://backend-service:8080
# Automatically expands to backend-service.production.svc.cluster.local
```

**Cross-Namespace Resolution:**
```bash
# Must specify full name or at least service.namespace
curl http://backend-service.staging:8080
# Or fully qualified
curl http://backend-service.staging.svc.cluster.local:8080
```

**External Name Services:**
Map internal DNS names to external DNS names via CNAME records:

```yaml
apiVersion: v1
kind: Service
metadata:
  name: external-database
  namespace: production
spec:
  type: ExternalName
  externalName: prod-db.company.rds.amazonaws.com
```

Usage: `external-database.production.svc.cluster.local` resolves to the AWS RDS endpoint.

### CoreDNS Customization

Customize DNS behavior via CoreDNS ConfigMap:

```yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: coredns
  namespace: kube-system
data:
  Corefile: |
    .:53 {
        errors
        health {
           lameduck 5s
        }
        ready
        kubernetes cluster.local in-addr.arpa ip6.arpa {
           pods insecure
           fallthrough in-addr.arpa ip6.arpa
           ttl 30
        }
        prometheus :9153
        forward . /etc/resolv.conf {
           max_concurrent 1000
        }
        cache 30
        loop
        reload
        loadbalance
    }
    # Custom domain for internal company services
    company.internal:53 {
        errors
        cache 30
        forward . 10.0.0.10 10.0.0.11
    }
```

**Key Directives:**
- **kubernetes**: Handles cluster DNS records
- **forward**: Forwards unknown queries to upstream DNS (node's /etc/resolv.conf)
- **cache**: Caches responses to reduce latency
- **rewrite**: Can modify queries (e.g., rewrite internal domains)

### Debugging DNS Issues

Common resolution problems and diagnostics:

```bash
# Test DNS resolution from within cluster
kubectl run -it --rm debug --image=busybox:1.36 --restart=Never -- nslookup kubernetes.default

# Check if CoreDNS pods are running
kubectl get pods -n kube-system -l k8s-app=kube-dns

# View CoreDNS logs
kubectl logs -n kube-system -l k8s-app=kube-dns

# Check DNS service endpoints
kubectl get endpoints -n kube-system kube-dns

# Verify Pod DNS configuration
kubectl get pod mypod -o yaml | grep -A 5 dnsPolicy
```

## 15.3 Ingress Controllers

While Services expose applications within or externally via LoadBalancers, Ingress resources provide HTTP/HTTPS routing, SSL termination, and name-based virtual hosting. The Ingress Controller implements the actual routing logic.

### Ingress vs. LoadBalancer

**LoadBalancer Service:**
- One IP per Service
- Layer 4 (TCP/UDP) load balancing
- Expensive (each Service gets its own cloud load balancer)
- No routing rules, SSL termination, or host/path-based routing

**Ingress:**
- Single IP for multiple Services
- Layer 7 (HTTP/HTTPS) routing
- Host-based routing (`api.example.com` vs `www.example.com`)
- Path-based routing (`/api/*` vs `/static/*`)
- SSL termination (TLS certificates at entry point)
- Cost-effective (one cloud load balancer for many services)

### NGINX Ingress Controller (Most Common)

The NGINX Ingress Controller is the de facto standard, maintained by the Kubernetes community and NGINX Inc.

**Installation (Helm):**
```bash
helm repo add ingress-nginx https://kubernetes.github.io/ingress-nginx
helm repo update

helm install ingress-nginx ingress-nginx/ingress-nginx \
  --namespace ingress-nginx \
  --create-namespace \
  --set controller.service.type=LoadBalancer \
  --set controller.metrics.enabled=true \
  --set controller.podAnnotations."prometheus\.io/scrape"="true" \
  --set controller.podAnnotations."prometheus\.io/port"="10254"
```

**Basic Ingress Resource:**
```yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: example-ingress
  namespace: production
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /
    nginx.ingress.kubernetes.io/ssl-redirect: "true"
spec:
  ingressClassName: nginx  # Selects which controller implements this
  rules:
  - host: api.example.com
    http:
      paths:
      - path: /v1
        pathType: Prefix
        backend:
          service:
            name: api-service-v1
            port:
              number: 80
      - path: /v2
        pathType: Prefix
        backend:
          service:
            name: api-service-v2
            port:
              number: 80
  - host: www.example.com
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: frontend-service
            port:
              number: 80
  tls:
  - hosts:
    - api.example.com
    - www.example.com
    secretName: example-tls-secret  # References TLS Secret
```

**Path Types:**
- **Exact**: Matches URL path exactly (case-sensitive)
- **Prefix**: Matches URL path prefix split by `/` (e.g., `/api` matches `/api`, `/api/`, `/api/v1`)
- **ImplementationSpecific**: Interpretation depends on IngressClass (avoid for portability)

### Advanced Ingress Patterns

**Rate Limiting:**
```yaml
metadata:
  annotations:
    nginx.ingress.kubernetes.io/limit-rps: "10"  # Requests per second
    nginx.ingress.kubernetes.io/limit-connections: "5"
```

**Sticky Sessions (Session Affinity):**
```yaml
metadata:
  annotations:
    nginx.ingress.kubernetes.io/affinity: "cookie"
    nginx.ingress.kubernetes.io/session-cookie-name: "route"
    nginx.ingress.kubernetes.io/session-cookie-expires: "172800"
    nginx.ingress.kubernetes.io/session-cookie-max-age: "172800"
```

**CORS Configuration:**
```yaml
metadata:
  annotations:
    nginx.ingress.kubernetes.io/enable-cors: "true"
    nginx.ingress.kubernetes.io/cors-allow-origin: "https://example.com"
    nginx.ingress.kubernetes.io/cors-allow-methods: "GET, POST, OPTIONS"
```

**Canary Deployments:**
```yaml
metadata:
  annotations:
    nginx.ingress.kubernetes.io/canary: "true"
    nginx.ingress.kubernetes.io/canary-weight: "20"  # 20% traffic
    # or by header
    nginx.ingress.kubernetes.io/canary-by-header: "X-Canary"
    nginx.ingress.kubernetes.io/canary-by-header-value: "true"
```

### Alternative Ingress Controllers

**Traefik (Cloud-Native, Dynamic):**
- Auto-discovery of services
- Native Let's Encrypt support
- Middleware chains (authentication, rate limiting, redirects)
- Dashboard for visualization

**HAProxy Ingress (High Performance):**
- Known for performance and stability
- Advanced load balancing algorithms
- Suitable for high-throughput scenarios

**Contour (Envoy-Based):**
- Uses Lyft's Envoy proxy as data plane
- HTTPProxy CRD (alternative to Ingress) with more features
- Delegation patterns for multi-team environments

**Istio Ingress Gateway (Service Mesh):**
- Part of Istio service mesh
- Advanced traffic management (traffic splitting, mirroring)
- mTLS enforcement
- Detailed telemetry

## 15.4 Network Policies

By default, Kubernetes allows all Pods to communicate freely—a "zero trust" violation. NetworkPolicies provide micro-segmentation, controlling traffic flow at the IP address or port level (OSI layer 3/4).

### Network Policy Basics

**Default Behavior:**
- **Isolation**: Non-isolated Pods accept all traffic
- **Policy Selection**: Pods become isolated when any NetworkPolicy selects them
- **Direction**: Policies can restrict ingress (incoming), egress (outgoing), or both

**Prerequisites:**
NetworkPolicies require a CNI plugin that supports them (Calico, Cilium, Weave, Antrea). Flannel does **not** support NetworkPolicies.

### Policy Examples

**Deny All Ingress (Default Deny):**
Start with default deny, then explicitly allow required traffic:

```yaml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: default-deny-ingress
  namespace: production
spec:
  podSelector: {}  # Empty = all pods in namespace
  policyTypes:
  - Ingress
  # No ingress rules = deny all incoming
```

**Allow Specific Ingress:**
```yaml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: api-allow
  namespace: production
spec:
  podSelector:
    matchLabels:
      app: api
      tier: backend
  policyTypes:
  - Ingress
  ingress:
  - from:
    - namespaceSelector:
        matchLabels:
          name: frontend  # From namespace labeled 'frontend'
    - podSelector:
        matchLabels:
          app: web  # AND/OR from pods labeled app=web in same namespace
    ports:
    - protocol: TCP
      port: 8080
```

**Allow Specific Egress (External API Calls):**
```yaml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: egress-restricted
  namespace: production
spec:
  podSelector:
    matchLabels:
      app: payment-service
  policyTypes:
  - Egress
  egress:
  - to:
    - namespaceSelector:
        matchLabels:
          name: database
    ports:
    - protocol: TCP
      port: 5432
  - to:  # Allow DNS
    - namespaceSelector: {}
      podSelector:
        matchLabels:
          k8s-app: kube-dns
    ports:
    - protocol: UDP
      port: 53
  - to:  # Specific external API
    - ipBlock:
        cidr: 203.0.113.0/24
        except:
        - 203.0.113.5/32  # Exclude specific IP
    ports:
    - protocol: TCP
      port: 443
```

**Complex Multi-Rule Policy:**
```yaml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: comprehensive-policy
spec:
  podSelector:
    matchLabels:
      app: complex-app
  policyTypes:
  - Ingress
  - Egress
  ingress:
  - from:
    - ipBlock:
        cidr: 10.0.0.0/8
    - namespaceSelector:
        matchLabels:
          environment: trusted
    - podSelector:
        matchLabels:
          app: authorized-client
    ports:
    - protocol: TCP
      port: 80
    - protocol: TCP
      port: 443
  egress:
  - to:
    - podSelector:
        matchLabels:
          app: database
    ports:
    - protocol: TCP
      port: 5432
  - to:
    - podSelector:
        matchLabels:
          k8s-app: kube-dns
    ports:
    - protocol: UDP
      port: 53
```

### CNI-Specific Features

**Calico NetworkPolicy (Enhanced):**
Calico extends standard NetworkPolicy with additional features:

```yaml
apiVersion: projectcalico.org/v3
kind: NetworkPolicy
metadata:
  name: advanced-policy
spec:
  selector: app == 'backend'
  types:
  - Ingress
  ingress:
  - action: Allow
    protocol: TCP
    source:
      selector: app == 'frontend'
      namespaceSelector: project == 'production'
    destination:
      ports:
      - 8080
  - action: Deny
    protocol: TCP
    source:
      notSelector: trusted == 'true'
```

**Cilium (eBPF-Based):**
Cilium provides Layer 7 (HTTP) policies and identity-based security:

```yaml
apiVersion: cilium.io/v2
kind: CiliumNetworkPolicy
metadata:
  name: l7-policy
spec:
  endpointSelector:
    matchLabels:
      app: backend
  ingress:
  - fromEndpoints:
    - matchLabels:
        app: frontend
    toPorts:
    - ports:
      - port: "80"
        protocol: TCP
      rules:
        http:
        - method: "GET"
          path: "/api/v1/.*"
        - method: "POST"
          path: "/api/v1/data"
```

## 15.5 CNI Plugins

The Container Network Interface (CNI) is a specification and libraries for configuring network interfaces in Linux containers. Kubernetes delegates Pod networking to CNI plugins, which handle IP allocation, connectivity, and policy enforcement.

### Major CNI Plugins

**Calico (Most Feature-Rich):**
- **Routing**: BGP (Border Gateway Protocol) for direct routing, or IPIP/VXLAN overlays
- **Policy**: Comprehensive NetworkPolicy support with global policies
- **IPAM**: Flexible IP pool management
- **eBPF**: Optional eBPF data plane for performance
- **Use Case**: Enterprise environments requiring fine-grained security and on-premises networking

**Cilium (eBPF-Native):**
- **Technology**: Uses Linux eBPF for kernel-level packet processing
- **Observability**: Hubble for network flow visibility
- **Security**: Identity-based security (not just IP-based), Layer 7 policies
- **Service Mesh**: Can replace Istio sidecars with eBPF (better performance)
- **Use Case**: Security-focused environments, high-performance networking, cloud-native observability

**Flannel (Simplest):**
- **Overlay**: VXLAN or UDP overlay networks
- **Simplicity**: Easy setup, minimal configuration
- **Limitations**: No NetworkPolicy support (requires Canal: Flannel + Calico policy)
- **Use Case**: Small clusters, development environments, basic connectivity needs

**Weave Net:**
- **Mesh**: Peer-to-peer mesh networking
- **Encryption**: Optional encryption between nodes
- **Discovery**: Automatic peer discovery
- **Use Case**: Multi-cloud deployments, dynamic clusters

**Antrea (VMware):**
- **Integration**: Native NSX-T integration for VMware environments
- **eBPF**: Optional eBPF acceleration
- **Windows**: Strong Windows node support
- **Use Case**: Hybrid VMware/Kubernetes environments

### CNI Selection Criteria

| Feature | Calico | Cilium | Flannel | Weave | Antrea |
|---------|--------|--------|---------|-------|--------|
| **Network Policies** | Yes | Yes (L7) | No | Yes | Yes |
| **Overlay/Underlay** | Both | Both | Overlay | Mesh | Both |
| **Encryption** | WireGuard/IPsec | WireGuard/IPsec | No | Yes | IPsec |
| **eBPF Support** | Optional | Native | No | No | Optional |
| **Windows Support** | Limited | Limited | No | Yes | Yes |
| **Service Mesh** | No | Yes (partial) | No | No | No |
| **Complexity** | Medium | High | Low | Low | Medium |

### CNI Configuration

**Calico Installation:**
```bash
kubectl apply -f https://raw.githubusercontent.com/projectcalico/calico/v3.26.0/manifests/calico.yaml

# Verify
kubectl get pods -n kube-system -l k8s-app=calico-node
```

**Cilium Installation:**
```bash
helm repo add cilium https://helm.cilium.io/
helm install cilium cilium/cilium --namespace kube-system \
  --set hubble.enabled=true \
  --set hubble.relay.enabled=true \
  --set hubble.ui.enabled=true
```

**Flannel Installation:**
```bash
kubectl apply -f https://github.com/flannel-io/flannel/releases/latest/download/kube-flannel.yml
```

## 15.6 Service Mesh Introduction

While Kubernetes provides basic service-to-service communication through Services, Service Meshes add advanced traffic management, security, and observability without modifying application code.

**Service Mesh Architecture:**
- **Data Plane**: Sidecar proxies (Envoy, Linkerd-proxy) injected into application Pods
- **Control Plane**: Management plane configuring proxies and collecting telemetry
- **Sidecar Pattern**: Proxy intercepts all network traffic to/from the main application container

**Key Capabilities:**
- **mTLS**: Automatic mutual TLS encryption between services
- **Traffic Splitting**: Canary deployments, A/B testing, blue/green deployments
- **Retries/Timeouts**: Configurable resilience patterns
- **Observability**: Distributed tracing, metrics, service graphs
- **Authorization**: Service-to-service access control (L7 policies)

### Istio (Most Feature-Rich)

**Architecture:**
- **istiod**: Combined control plane (Pilot, Citadel, Galley)
- **Envoy Sidecars**: Data plane proxies injected via webhook

**Installation:**
```bash
istioctl install --set profile=default -y
kubectl label namespace default istio-injection=enabled
```

**Traffic Management Example:**
```yaml
apiVersion: networking.istio.io/v1beta1
kind: VirtualService
metadata:
  name: reviews-route
spec:
  hosts:
  - reviews
  http:
  - match:
    - headers:
        end-user:
          exact: jason
    route:
    - destination:
        host: reviews
        subset: v2
  - route:
    - destination:
        host: reviews
        subset: v1
      weight: 90
    - destination:
        host: reviews
        subset: v2
      weight: 10
```

### Linkerd (Lightweight Alternative)

- **Resource Usage**: Significantly lighter than Istio
- **Simplicity**: Easier to operate, opinionated defaults
- **Performance**: Rust-based proxy (linkerd2-proxy) optimized for latency

**Installation:**
```bash
linkerd install | kubectl apply -f -
linkerd viz install | kubectl apply -f -  # Dashboard
linkerd inject deployment.yml | kubectl apply -f -
```

### When to Use Service Mesh

**Use Service Mesh when:**
- You require mTLS between services (zero-trust networking)
- You need sophisticated traffic management (canary, circuit breaking)
- You lack application-level observability and need distributed tracing
- You run hundreds of microservices requiring unified policy enforcement

**Avoid Service Mesh when:**
- Cluster has < 20 services (overhead exceeds benefit)
- Network latency is critical (sidecars add ~3-5ms)
- Team lacks operational capacity to manage complex control planes

## 15.7 External DNS

ExternalDNS automates the creation of DNS records in external DNS providers (Route53, Cloudflare, Google Cloud DNS) based on Kubernetes Ingress and Service resources.

**Architecture:**
- Watches Kubernetes API for Services/Ingresses with specific annotations
- Creates/updates DNS records in configured provider
- Removes records when Kubernetes resources are deleted

**Installation (AWS Route53):**
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: external-dns
  namespace: kube-system
spec:
  strategy:
    type: Recreate
  selector:
    matchLabels:
      app: external-dns
  template:
    metadata:
      labels:
        app: external-dns
    spec:
      serviceAccountName: external-dns
      containers:
      - name: external-dns
        image: registry.k8s.io/external-dns/external-dns:v0.14.0
        args:
        - --source=service
        - --source=ingress
        - --domain-filter=example.com
        - --provider=aws
        - --policy=upsert-only
        - --aws-zone-type=public
        - --registry=txt
        - --txt-owner-id=kubernetes-cluster-1
```

**Usage with Ingress:**
```yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: example-ingress
  annotations:
    external-dns.alpha.kubernetes.io/hostname: app.example.com
    external-dns.alpha.kubernetes.io/ttl: "300"
spec:
  rules:
  - host: app.example.com
    http:
      paths:
      - path: /
        backend:
          service:
            name: my-service
            port:
              number: 80
```

## 15.8 TLS/SSL Management

Encrypting traffic in transit is mandatory for production workloads. Kubernetes provides mechanisms for TLS termination at Ingress and Pod-to-Pod encryption via Service Mesh.

### cert-manager

cert-manager automates TLS certificate provisioning and renewal from Let's Encrypt, Vault, or private CAs.

**Installation:**
```bash
helm repo add jetstack https://charts.jetstack.io
helm repo update
helm install cert-manager jetstack/cert-manager \
  --namespace cert-manager \
  --create-namespace \
  --set installCRDs=true
```

**Issuer Configuration (Let's Encrypt):**

```yaml
apiVersion: cert-manager.io/v1
kind: ClusterIssuer
metadata:
  name: letsencrypt-prod
spec:
  acme:
    server: https://acme-v02.api.letsencrypt.org/directory
    email: admin@example.com
    privateKeySecretRef:
      name: letsencrypt-prod
    solvers:
    - http01:
        ingress:
          class: nginx
      selector:
        dnsZones:
        - "example.com"
```

**Certificate Resource:**
```yaml
apiVersion: cert-manager.io/v1
kind: Certificate
metadata:
  name: example-com
  namespace: production
spec:
  secretName: example-com-tls
  issuerRef:
    name: letsencrypt-prod
    kind: ClusterIssuer
  dnsNames:
  - example.com
  - www.example.com
  renewBefore: 720h  # Renew 30 days before expiry
  duration: 2160h    # 90 days (Let's Encrypt default)
```

**Ingress Integration (Automatic):**
```yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: example-ingress
  annotations:
    cert-manager.io/cluster-issuer: "letsencrypt-prod"
spec:
  tls:
  - hosts:
    - example.com
    secretName: example-com-tls  # Auto-created by cert-manager
  rules:
  - host: example.com
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: my-service
            port:
              number: 80
```

### TLS at Pod Level (Service Mesh)

For end-to-end encryption terminating at the Pod rather than Ingress:

```yaml
apiVersion: security.istio.io/v1beta1
kind: PeerAuthentication
metadata:
  name: default
  namespace: production
spec:
  mtls:
    mode: STRICT  # Require mTLS for all services in namespace
```

### Certificate Rotation

**Manual Rotation:**
```bash
# Delete secret to trigger re-issuance
kubectl delete secret example-com-tls -n production
# cert-manager will recreate immediately
```

**Monitoring:**
```bash
# Check certificate status
kubectl describe certificate example-com -n production

# Check certificate expiry
kubectl get certificate -n production -o jsonpath='{range .items[*]}{.metadata.name}{"\t"}{.status.notAfter}{"\n"}{end}'
```

---

## Chapter Summary and Preview

In this chapter, we explored the Kubernetes networking stack from fundamental Pod connectivity through sophisticated traffic management. We established that Kubernetes mandates a flat network model where every Pod receives a unique routable IP without NAT, implemented through CNI plugins such as Calico (BGP/policy-rich), Cilium (eBPF/observability), or Flannel (simplicity). CoreDNS provides cluster-internal service discovery via DNS records for Services, while Ingress Controllers (NGINX, Traefik) extend this to external HTTP/HTTPS routing with SSL termination, path-based routing, and canary capabilities. NetworkPolicies enforce micro-segmentation at Layer 3/4, critical for zero-trust security postures. We introduced Service Mesh concepts (Istio, Linkerd) for advanced traffic management and mTLS, ExternalDNS for automated public DNS integration, and cert-manager for automated TLS certificate lifecycle management.

**Key Takeaways:**
- Choose CNI plugins based on requirements: Flannel for simplicity, Calico for policy enforcement, Cilium for observability and Layer 7 policies.
- Always implement default-deny NetworkPolicies and explicitly allow required traffic; unspecified policies allow all traffic.
- Use Ingress rather than individual LoadBalancer Services to minimize cloud costs and enable sophisticated routing.
- Implement cert-manager for automated Let's Encrypt integration; avoid manual certificate management.
- Service Meshes provide powerful capabilities but add complexity and latency; adopt only when microservices count justifies overhead.

**Next Chapter Preview:**
Chapter 16: Storage and Volumes addresses stateful workloads, exploring PersistentVolumes (PV) and PersistentVolumeClaims (PVC) for durable storage, StorageClasses for dynamic provisioning, StatefulSets for ordered deployment with stable network identity, and database persistence strategies. You will learn to distinguish ephemeral storage (emptyDir) from persistent storage, implement backup strategies for stateful applications, and manage storage across node failures—essential capabilities for running databases, message queues, and file stores in Kubernetes.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='14. kubernetes_core_resources.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='16. storage_and_volumes.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
